# Mini Project: Build a Machine Learning Model

## Predict Total Fare on the NYC Taxi Dataset

Welcome to the NYC Taxi Fare Prediction project! In this Colab, we will continue using the NYC Taxi Dataset to predict the fare amount for taxi rides using a subset of available features. We will go through three main stages: building a baseline model, creating a full model, and performing hyperparameter tuning to enhance our predictions.

Now that you've completed exploratory data analysis on this dataset you should have a good understanding of the feature space.

## Project Objectives

The primary objectives of this project are as follows:

Baseline Model: We will start by building a simple baseline model to establish a benchmark for our predictions. This model will serve as a starting point to compare the performance of our subsequent models.

Full Model: Next, we will develop a more comprehensive model that leverages machine learning techniques to improve prediction accuracy. We will use Scikit-Learn's model pipeline to build a framework that enables rapid experimentation.

Hyperparameter Tuning: Lastly, we will optimize our full model by fine-tuning its hyperparameters. By systematically adjusting the parameters that control model behavior, we aim to achieve the best possible performance for our prediction task.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

Load the NYC taxi dataset into a Pandas DataFrame and do a few basic checks to ensure the data is loaded properly. Note, there are several months of data that can be used. For simplicity, use the Yellow Taxi 2022-01 parquet file [here](https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet). Here are your tasks:

  1. Load the `yellow_tripdata_2022-01.parquet` file into Pandas.
  2. Print the first 5 rows of data.
  3. Drop any rows of data that contain NULL values.
  4. Create a new feature, 'trip_duration' that captures the duration of the trip in minutes.
  5. Create a varible named 'target_variable' to store the name of the thing we're trying to predict, 'total_amount'.
  6. Create a list called 'feature_cols' containing the feature names that we'll be using to predict our target variable. The list should contain 'VendorID', 'trip_distance', 'payment_type', 'PULocationID', 'DOLocationID', and 'trip_duration'.

In [2]:
# Load the dataset into a pandas DataFrame (from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

df = pd.read_parquet(" https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet")


In [3]:
# Display the first few rows of the dataset

# Note that df.head() dosplays the first 5 rows by default.
print(df.head())

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2022-01-01 00:35:40   2022-01-01 00:53:29              2.0   
1         1  2022-01-01 00:33:43   2022-01-01 00:42:07              1.0   
2         2  2022-01-01 00:53:21   2022-01-01 01:02:19              1.0   
3         2  2022-01-01 00:25:21   2022-01-01 00:35:23              1.0   
4         2  2022-01-01 00:36:48   2022-01-01 01:14:20              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           3.80         1.0                  N           142           236   
1           2.10         1.0                  N           236            42   
2           0.97         1.0                  N           166           166   
3           1.09         1.0                  N           114            68   
4           4.30         1.0                  N            68           163   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [4]:
# Drop rows with missing values.

# Check number of null values prior to dropping.
df.isnull().sum()
# Drop null values, using the same dataframe.
df.dropna(inplace=True)
# Check number of null values after dropping.
df.isnull().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
congestion_surcharge     0
airport_fee              0
dtype: int64

In [5]:
# Create new feature, 'trip_duration'.

# 4. Create a new feature, 'trip_duration' that captures the duration of the trip in minutes.
df["trip_duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60
# Note that the time_duration column contains a float the represents the minutes. For example, 17 minute 30 second trip would be 17.5 minutes.
df["trip_duration"].head()

0    17.816667
1     8.400000
2     8.966667
3    10.033333
4    37.533333
Name: trip_duration, dtype: float64

In [6]:
# 5. Create a varible named 'target_variable' to store the name of the thing we're trying to predict, 'total_amount'.
df["target_variable"] = df["total_amount"]
print(df["target_variable"].head())
print(df["total_amount"].head())

0    21.95
1    13.30
2    10.56
3    11.80
4    30.30
Name: target_variable, dtype: float64
0    21.95
1    13.30
2    10.56
3    11.80
4    30.30
Name: total_amount, dtype: float64


In [7]:
# Create a list called feature_col to store column names

# 6. Create a list called 'feature_cols' containing the feature names that we'll be using to predict our target variable.
# The list should contain 'VendorID', 'trip_distance', 'payment_type', 'PULocationID', 'DOLocationID', and 'trip_duration'.
feature_cols = ["VendorID", "trip_distance", "payment_type", "PULocationID", "DOLocationID", "trip_duration"]
feature_cols

['VendorID',
 'trip_distance',
 'payment_type',
 'PULocationID',
 'DOLocationID',
 'trip_duration']

Splitting a dataset into training and test sets is a crucial step in machine learning model development. It allows us to evaluate the performance and generalization ability of our models accurately. The training set is used to train the model, while the test set serves as an independent sample for evaluating its performance.

1. **Model Training**: The training set is used to fit the model, allowing it to learn the underlying patterns and relationships between the features and the target variable. By exposing the model to a diverse range of examples in the training set, it can capture the underlying structure of the data.

2. **Model Evaluation**: The test set, which is independent of the training set, is crucial for evaluating how well the trained model generalizes to unseen data. It provides an unbiased assessment of the model's performance on new instances. By measuring the model's accuracy, precision, recall, or other evaluation metrics on the test set, we can estimate how well the model will perform on unseen data.

3. **Preventing Overfitting**: Overfitting occurs when a model learns the training data's noise and idiosyncrasies instead of the underlying patterns. By evaluating the model on the test set, we can identify if the model is overfitting. If the model performs significantly worse on the test set compared to the training set, it indicates overfitting. In such cases, we might need to adjust the model, feature selection, or regularization techniques to improve generalization.

4. **Hyperparameter Tuning**: Splitting the dataset allows us to perform hyperparameter tuning on the model. Hyperparameters are configuration settings that control the learning process, such as learning rate, regularization strength, or the number of hidden layers in a neural network. By using a validation set (often created from a portion of the training set), we can iteratively adjust the hyperparameters and select the best combination that maximizes the model's performance on the validation set. The final evaluation on the test set provides an unbiased estimate of the model's performance.

By splitting the dataset into training and test sets, we can ensure that our models are both well-trained and accurately evaluated. This separation helps us understand how the model will perform on new, unseen data, which is critical for assessing its effectiveness and making informed decisions about its deployment.

Here is your task:

  1. Use Scikit-Learn's [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split the data into training and test sets. Don't forget to set the random state.

In [8]:
# Split dataset into training and test sets

# Need to create x and y data
# Use the list of feature columns to create x data for features.
x = df[feature_cols]
print(x.shape, type(x))
# Use target_variable as the y data.
y = df["target_variable"]
print(y.shape, type(y))
# Set seed for reproducability.
seed = 123
# Can also set size to determine amount of data to use in the test split. Default is 0.25 (left it at default).
X_train, X_test, y_train, y_test = train_test_split(x, y.values, random_state=seed)

(2392428, 6) <class 'pandas.core.frame.DataFrame'>
(2392428,) <class 'pandas.core.series.Series'>


The importance of a baseline model, even if it uses a simple strategy like always predicting the mean, cannot be understated. Here's why a baseline model is valuable:

1. **Performance Comparison**: A baseline model serves as a reference point for evaluating the performance of more sophisticated models. By establishing a simple yet reasonable baseline, we can determine whether our advanced models offer any significant improvement over this basic approach. It helps us set realistic expectations and gauge the effectiveness of our efforts.

2. **Model Complexity**: A baseline model provides insight into the complexity required to solve the prediction task. If a simple strategy like predicting the median performs reasonably well, it suggests that the problem might not necessitate complex modeling techniques. Conversely, if the baseline model performs poorly, it indicates the presence of more intricate patterns that need to be captured by more sophisticated models.

3. **Minimum Performance Requirement**: A baseline model can establish a minimum performance requirement for a predictive task. If we cannot outperform the baseline, it suggests that our models have failed to capture even the most fundamental relationships within the data. In such cases, we may need to revisit our data preprocessing steps, feature engineering techniques, or consider other external factors affecting the task.

4. **Identifying Data Issues**: A baseline model can help identify potential issues within the dataset. If the baseline model performs poorly, it may indicate problems like missing values, outliers, or data inconsistencies. These issues can be further investigated and resolved to improve the overall model performance.

While a baseline model like always predicting the median may not offer the highest prediction accuracy, its importance lies in its role as a starting point for model development and evaluation. It provides a solid foundation for comparing and assessing the performance of more complex models, ensuring that any improvements made are meaningful and significant.

Here is your task:

  1. Create a model that always predicts the mean total fare of the training dataset. Use Scikit-Learn's [mean_absolute_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) to evaluate this model. Is it any good?

In [30]:
# Create a baseline for mean absolute error of total amount

# sklearn.dummy.DummyRegressor can be used for a simple baseline.
from sklearn.dummy import DummyRegressor

# strategy is used to generate predictions. Per the documentation, "mean" always predicts the mean of the training set.
baseline_model = DummyRegressor(strategy="mean")
# Fit the simple model.
baseline_mean = baseline_model.fit(X_train, y_train)
#  Create prediction.
y_predicted = baseline_model.predict(X_test)

# Mean absolute error compares actual to predicted.
baseline_model_mae = mean_absolute_error(y_test, y_predicted)
print("Baseline model mean absolute error:", baseline_model_mae)
print("Taxi total amount mean:", df["target_variable"].mean() )

Baseline model mean absolute error: 9.200721118066248
Taxi total amount mean: 19.02453219908813


1. Create a model that always predicts the mean total fare of the training dataset. Use Scikit-Learn's mean_absolute_error to evaluate this model. Is it any good?<br>
Mean absolute error is the mean of the absolute errors. The absolute error is the absolute value of the difference between the forecasted value and the actual value. In the given context of NYC taxi fares, an absolute average error of 9.20 is should be able to be improved upon significantly, especially since the mean of the total amount is $19.02 (9.20/19.02 = 48%).Expect models that perform better to have a lower mean absolute error.<br>

With a baseline metric in place, we can try to build a machine learning model. Obviously, if the model can't beat the baseline then there are some major issues to be resolved.

It's always a good idea to start with a simple machine learning model, like linear regression, and build upon it if necessary.

Here are your tasks:

  1. Use Scikit-Learn's [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) to preprocess the categorical and continuous features independently. Apply the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to the continuous columns and [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to the categorical columns.

  One-hot encoding is a popular technique used to represent categorical variables numerically in machine learning models. It transforms categorical features into a binary vector representation, where each category is represented by a binary column. Here's an explanation of one-hot encoding:

  When working with categorical variables, such as colors (e.g., red, blue, green) or vehicle types (e.g., car, truck, motorcycle), machine learning algorithms often require numerical inputs. However, directly assigning numerical values to categories can introduce unintended relationships or orderings between them. For example, assigning the values 0, 1, and 2 to the categories red, blue, and green may imply a sequential relationship, which is not desired.

  One-hot encoding solves this problem by creating new binary columns, equal to the number of unique categories in the original feature. Each binary column represents a specific category and takes a value of 1 if the data point belongs to that category, and 0 otherwise. This encoding ensures that no implicit ordering or relationship exists between the categories.

  2. Integrate the preprocessor in the previous step with Scikit-Learn's [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) model using a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

  3. Train the pipeline on the training data.

  4. Evaluate the model using mean absolute error as a metric on the test data. Does the model beat the baseline?


In [10]:
# Identify continuous and categorical columns from the features list.
print(df[feature_cols].head())

   VendorID  trip_distance  payment_type  PULocationID  DOLocationID  \
0         1           3.80             1           142           236   
1         1           2.10             1           236            42   
2         2           0.97             1           166           166   
3         2           1.09             2           114            68   
4         2           4.30             1            68           163   

   trip_duration  
0      17.816667  
1       8.400000  
2       8.966667  
3      10.033333  
4      37.533333  


Continuous: trip_distance, trip_duration.<br>
Categorical: VendorID, payment_type<br>

In [11]:
# Use Scikit-Learn's ColumnTransformer to preprocess the categorical and
# continuous features independently.

# Continuous features transformation with StandardScaler.
# Categorical features transformation with OneHotEncoder.
#"VendorID",
preprocessor =ColumnTransformer(transformers=[("continuous", StandardScaler() , ["trip_distance", "trip_duration"] ), ("categorical", OneHotEncoder(), ["payment_type"])])

In [12]:
# Create a pipeline object containing the column transformations and regression
# model.

# Create the model
lr_model = LinearRegression()
# Create the Pipeline
pipeline = Pipeline(steps=[("transformer", preprocessor), ("model", lr_model)])

In [13]:
# Fit the pipeline on the training data.

pipeline.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('continuous',
                                                  StandardScaler(),
                                                  ['trip_distance',
                                                   'trip_duration']),
                                                 ('categorical',
                                                  OneHotEncoder(),
                                                  ['payment_type'])])),
                ('model', LinearRegression())])

In [14]:
# Make predictions on the test data.

y_predicted = pipeline.predict(X_test)

In [15]:
# 4. Evaluate the model using mean absolute error as a metric on the test data.
lr_model_mae = mean_absolute_error(y_test, y_predicted)
print(lr_model_mae)

2.986181060483021


4. Does the model beat the baseline?<br>
Yes, the MAE is significantly lower for the liner regression model (2.99) than for the baseline model (9.20). This is expected given the simple nature of the baseline model (DummyRegression of the mean) and the preprocessing done (both continuous and categorical variables) combined with the more sophisticated linear regression model actually trying to fit a line.<br>

Random Forest Regression and Linear Regression are two commonly used regression algorithms, each with its own advantages and suitability for different scenarios. Random Forest Regression offers several advantages over Linear Regression, including:

1. **Non-linearity**: Random Forest Regressor is capable of capturing non-linear relationships between features and the target variable. In contrast, Linear Regression assumes a linear relationship between the features and the target. When faced with non-linear relationships or complex feature interactions, Random Forest Regressor can provide more accurate predictions.

2. **Robustness to Outliers**: Random Forest Regressor is generally more robust to outliers compared to Linear Regression. Outliers can disproportionately impact the coefficients and predictions of Linear Regression models. However, as an ensemble of decision trees, Random Forest Regressor can mitigate the effect of outliers by averaging predictions from multiple trees.

3. **Feature Importance**: Random Forest Regressor provides a measure of feature importance, which helps identify the most influential features for making predictions. This information is useful for feature selection, understanding the underlying relationships in the data, and gaining insights into the problem domain. Unlike Linear Regression, which provides coefficient values indicating the direction and magnitude of relationships, Random Forest Regressor explicitly highlights feature importance.

4. **Handling of Categorical Variables**: Random Forest Regressor can effectively handle categorical variables without requiring pre-processing steps like one-hot encoding. It can directly incorporate categorical variables into the model, making it more convenient when working with mixed data types. In contrast, Linear Regression often requires categorical variables to be encoded or transformed before use.

5. **Handling of High-Dimensional Data**: Random Forest Regressor can handle datasets with a large number of features (high dimensionality) by automatically selecting subsets of features during the construction of individual decision trees. This reduces the risk of overfitting, which is a concern with Linear Regression when dealing with high-dimensional data.

6. **Resistance to Multicollinearity**: Random Forest Regressor is less affected by multicollinearity, which occurs when predictor variables are highly correlated. In Linear Regression, highly correlated features can lead to unstable coefficient estimates, making it challenging to interpret the individual effects of each feature. Random Forest Regressor, as an ensemble approach, is less impacted by multicollinearity because each tree is built independently.

Here are your tasks:

  1. Build a Random Forest Regressor model using Scikit-Learn's [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) and train it on the train data.

  2. Evaluate the performance of the model on the test data using mean absolute error as a metric. Mess around with various input parameter configurations to see how they affect the model. Can you beat the performance of the linear regression model?

In [16]:
# Build random forest regressor model

# Set the seed for random state for reproducability.
#rfr_model1 = RandomForestRegressor(random_state=seed)
# Model 2 takes 7-8 minutes to run (adjusted max depth per notes below).
rfr_model2 = RandomForestRegressor(max_depth=8,random_state=seed)
# Train it on the training data.
#rfr_model1.fit(X_train, y_train)
rfr_model2.fit(X_train, y_train)

RandomForestRegressor(max_depth=8, random_state=123)

Initial model (model1) ran into memory/timeout issue after trying to fit for more than 10 minutes. For model2, set max_depth=8 to avoid memory/timeout issue. <br>

In [17]:
# Make predictions on the test data

y_predicted = rfr_model2.predict(X_test)


In [18]:
# 2. Evaluate the performance of the model on the test data using mean absolute error as a metric.

rfr_model2_mae = mean_absolute_error(y_test, y_predicted)
print(rfr_model2_mae)



2.6899780635695647


model2 MAE of 2.69 is the best so far (better than Linear Regression model 2.99 MAE)<br>

2. Mess around with various input parameter configurations to see how they affect the model. Can you beat the performance of the linear regression model?<br>
RandomForestRegressor model #2 already beats the linear regression (2.69 MAE vs. 3.01). Will try adjusting other parameters (n_estimators, criterion,
 max_depth) to see if MAE improves.

In [19]:
# Get the current parameters of the model (model2).
print(rfr_model2.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 8, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0, 'warm_start': False}


In [20]:
# Try different numbers of estimators. Default value is 100.
rfr_model3 = RandomForestRegressor(n_estimators=50, max_depth=8,random_state=seed)
# Train it on the training data.
# Takes about 3-4 minutes.
rfr_model3.fit(X_train, y_train)

# Make predictions on the test data
y_predicted = rfr_model3.predict(X_test)

# Evaluate the performance of the model on the test data using mean absolute error as a metric.
rfr_model3_mae = mean_absolute_error(y_test, y_predicted)
print(rfr_model3_mae)

2.6955469982224605


Based on MAE, model 3 (2.70) performs a little worse than model 2 (2.69), but still better than model 1 Linear Regression (2.99).<br>

In [21]:
# Try different numbers of estimators. Default value is 100.
rfr_model4 = RandomForestRegressor(n_estimators=200, max_depth=8,random_state=seed)
# Train it on the training data.
# Takes about 14 minutes.
rfr_model4.fit(X_train, y_train)

# Make predictions on the test data
y_predicted = rfr_model4.predict(X_test)

# Evaluate the performance of the model on the test data using mean absolute error as a metric.
rfr_model4_mae = mean_absolute_error(y_test, y_predicted)
print(rfr_model4_mae)

2.726725818448854


Based on MAE, model 4 (2.70) performs a worse than model 2 (2.69), but still better than model 1 Linear Regression (2.99).<br>
Will try adjusting criterion next.<br>

In [22]:
# Try using mean absolute error to decide splits. Default value is mean squared error.

# See note below regaridng commenting out of this section.
# Note: The fitting of this model takes about 3 hours to run.
#rfr_model5 = RandomForestRegressor(criterion="absolute_error", max_depth=8, random_state=seed)
# Train it on the training data.
#rfr_model5.fit(X_train, y_train)

# Make predictions on the test data
#y_predicted = rfr_model5.predict(X_test)

# Evaluate the performance of the model on the test data using mean absolute error as a metric.
#rfr_model5_mae = mean_absolute_error(y_test, y_predicted)
#print(rfr_model5_mae)

MAE of model 5 was 2.72, which was not as good as MAE for model 2 (2.69).<br>

Since optimizing max_depth is part of the next exercise, will leave the max_depth unchanged for now.<br>

Hyperparameter tuning plays a critical role in machine learning model development. It involves selecting the optimal values for the hyperparameters, which are configuration settings that control the behavior of the learning algorithm. Here's why hyperparameter tuning is so important in ML:

1. **Optimizing Model Performance**: The choice of hyperparameters can significantly impact the model's performance. By fine-tuning the hyperparameters, we can improve the model's accuracy, precision, recall, or other performance metrics. It helps to extract the maximum predictive power from the chosen algorithm and ensures that the model is well-suited to the specific problem at hand.

2. **Avoiding Overfitting and Underfitting**: Hyperparameter tuning helps strike a balance between overfitting and underfitting.

3. **Exploring Model Complexity**: Hyperparameter tuning enables us to explore the complexity of the model. For instance, in algorithms like decision trees or neural networks, we can adjust the number of layers, the number of neurons, or the maximum depth of the tree. By systematically modifying these hyperparameters, we can understand how different levels of complexity impact the model's performance and find the right balance between simplicity and complexity.

Note, there are multiple approaches to hyperparemeter tuning.  

While grid search is the easiest to understand and implement there are many advantages of Bayesian search over grid search for hyperparameter tuning:

1. **Efficiency**: Bayesian search is generally more efficient than grid search. Grid search explores all possible combinations of hyperparameter values, which can be computationally expensive and time-consuming, especially when dealing with a large number of hyperparameters or a wide range of values. Bayesian search, on the other hand, intelligently selects the next hyperparameter configuration to evaluate based on the results of previous evaluations. It focuses on areas of the hyperparameter space that are more likely to yield better performance, reducing the number of evaluations needed.

2. **Flexibility**: Bayesian search is flexible in handling continuous and discrete hyperparameters. It can handle both types of hyperparameters naturally and effectively. In contrast, grid search is more suitable for discrete hyperparameters but may struggle with continuous ones, as it requires discretization or defining a finite set of values to search over.

3. **Adaptive Search**: Bayesian search adapts its search strategy based on the results of previous evaluations. It maintains a probability distribution over the hyperparameter space, updating it with each evaluation. This allows it to dynamically allocate more evaluations to promising regions and explore unexplored areas. In contrast, grid search follows a fixed and predefined search grid, regardless of the results of previous evaluations.

4. **Better Convergence**: Bayesian search has the potential to converge to the optimal hyperparameter configuration more quickly.

Here are your tasks:

  1. Perform a grid-search on a Random Forest Regressor model. Only search the space for the parameters 'n_estimators', 'max_depth', and 'min_samples_split'. Note, this can take some time to run. Make sure you set reasonable boundaries for the search space. Use Scikit-Learn's [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) method.

  2. After you've identified the best parameters, train a random forest regression model using these parameters on the full training data.

  3. Evaluate the model from the previous step using the test data. How does your model perform?

In [23]:
# Define the hyperparameters to tune.

# 1. Only search the space for the parameters 'n_estimators', 'max_depth', and 'min_samples_split'.
# Define dictionary with parameters and values.
parameter_grid = {"n_estimators": [100, 150] , "max_depth": [3, 5, 7], "min_samples_split": [2, 3, 5] }

'n_estimators', 'max_depth', and 'min_samples_split':<br>
n_estimators: Already tried 50 and 200, so chose the default 100 and 150 to cover regular intervals between 50 and 200. Also, the default 100 produces the best result so far, so wanted to include it to explore further optimizations that could make the MAE with n_estimators=100 even lower. <br>
max_depth: In previous iteration of the model, max_depth=8 avoided athe memort error enocuntered with the default max_depth (None). Decide to try values (3, 5, 7) based on research as previously noted.<br>
min_samples_split: Chose the default value, and two values greater than it (3,5).<br>

In [24]:
# Perform grid search to find the best hyperparameters. This could take a while.

# Reduced the number of samples to decrease search time. Took about 4-5 hours to run.
grid_rfr_class = GridSearchCV(estimator=RandomForestRegressor(random_state=123), param_grid=parameter_grid, scoring="neg_mean_absolute_error", cv=5)
grid_rfr_class.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=123),
             param_grid={'max_depth': [3, 5, 7], 'min_samples_split': [2, 3, 5],
                         'n_estimators': [100, 150]},
             scoring='neg_mean_absolute_error')

In [25]:
# Get the best model and its parameters.

# Get the mest model.
best_rfr_model = grid_rfr_class.best_estimator_
print(best_rfr_model)
print(grid_rfr_class.best_params_)

RandomForestRegressor(max_depth=7, min_samples_split=5, n_estimators=150,
                      random_state=123)
{'max_depth': 7, 'min_samples_split': 5, 'n_estimators': 150}


In [26]:
# Fit the best classifier on the training data.
best_rfr_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=7, min_samples_split=5, n_estimators=150,
                      random_state=123)

In [27]:
# Make predictions on the test data

y_predicted = best_rfr_model.predict(X_test)

In [29]:
# Make predictions on the test data

# Get the MAE.
best_rfr_model_MAE = mean_absolute_error(y_test, y_predicted)
print(best_rfr_model_MAE)

2.6127182002885285


The MAE from the best model of the grid search (2.61) is better than the MAE from the linear regression (2.99) and the initial model2 random forest regressor (2.69). Given the proportion of the grid search model MAE to the mean of the total amount (2.61/19.02453 = 13.7%), additional exploration of the random forest and other models may result in an even lower MAE.